In [ ]:

import findspark
findspark.init()

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FreshWorks Data Engineer").master("local[*]").getOrCreate()

24/02/27 00:23:15 WARN Utils: Your hostname, ajith-Lenovo-G50-80 resolves to a loopback address: 127.0.1.1; using 192.168.1.40 instead (on interface wlp3s0)
24/02/27 00:23:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/02/27 00:23:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/27 00:23:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark

### 𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐏𝐫𝐢𝐜𝐞 𝐚𝐭 𝐬𝐭𝐚𝐫𝐭 𝐨𝐟 𝐭𝐡𝐞 𝐦𝐨𝐧𝐭𝐡 𝐚𝐧𝐝 𝐭𝐡𝐞 𝐝𝐢𝐟𝐟𝐞𝐫𝐞𝐧𝐜𝐞 𝐢𝐧 𝐭𝐡𝐞 𝐩𝐫𝐢𝐜𝐞 𝐟𝐫𝐨𝐦 𝐩𝐫𝐞𝐯𝐢𝐨𝐮𝐬 𝐦𝐨𝐧𝐭𝐡 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐬𝐤𝐮_𝐢𝐝

In [1]:
def create_dataframe():
    data = [
            (1, '2023-01-01', 10),
            (1, '2023-01-27', 13),
            (1, '2023-02-01', 14),
            (1, '2023-02-15', 15),
            (1, '2023-03-03', 18),
            (1, '2023-03-27', 15),
            (1, '2023-04-06', 20),
            (2, '2024-01-01', 50),
            (2, '2024-01-29', 100),
            (2, '2024-02-01', 150)
        ]
    
    schema = ["sku_id", "price_date", "price"]
    
    return spark.createDataFrame(data=data, schema=schema)

In [6]:
df = create_dataframe()

In [7]:
df.show()

+------+----------+-----+
|sku_id|price_date|price|
+------+----------+-----+
|     1|2023-01-01|   10|
|     1|2023-01-27|   13|
|     1|2023-02-01|   14|
|     1|2023-02-15|   15|
|     1|2023-03-03|   18|
|     1|2023-03-27|   15|
|     1|2023-04-06|   20|
|     2|2024-01-01|   50|
|     2|2024-01-29|  100|
|     2|2024-02-01|  150|
+------+----------+-----+



In [9]:
def solution(df):
    from pyspark.sql import Window
    from pyspark.sql.functions import row_number, desc, when, dayofmonth, date_format, add_months, col, last, lag
    
    # sort the 'sku_id' based on 'price_date' on Descending order
    sort_df = df.withColumn('row_num', row_number().over(Window.partitionBy('sku_id').orderBy(desc('price_date'))))

    # if day of the 'price_date' is > firstDay, then add next month to the 'price_date'. Else keep the same month and date as fristDay
    date_df = sort_df.withColumn('price_month', when(dayofmonth('price_date').__gt__(1), date_format(add_months('price_date', 1), 'yyyy-MM-01')) \
                                                     .otherwise(date_format('price_date', 'yyyy-MM-01'))
                                )
    
    # group by 'sku_id' and month start price
    group_df = date_df.groupBy(col('sku_id'), col('price_month').alias('price_date')).agg(last('price').alias('price'))
    
    # compare the price with previous month price and calculate the difference
    final_df = group_df.withColumn('price_diff', col('price')-lag('price').over(Window.partitionBy('sku_id').orderBy('price_date')))\
                        .orderBy('sku_id', 'price_date')
    
    return final_df

In [10]:
outDf = solution(df)
outDf.show()

+------+----------+-----+----------+
|sku_id|price_date|price|price_diff|
+------+----------+-----+----------+
|     1|2023-01-01|   10|      null|
|     1|2023-02-01|   14|         4|
|     1|2023-03-01|   15|         1|
|     1|2023-04-01|   15|         0|
|     1|2023-05-01|   20|         5|
|     2|2024-01-01|   50|      null|
|     2|2024-02-01|  150|       100|
+------+----------+-----+----------+

